In [3]:
import os
import itertools
import glob
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import openpyxl
import matplotlib.ticker
import matplotlib.pyplot as plt


sns.set(font_scale=1.2, style='whitegrid', font='CMU Sans Serif')

# # drawio colors
# # 189,215,238
# # 255,230,153
# pal = sns.color_palette(['#BDD7EE', '#FFE699'])
sns.set_palette("muted", 9)

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['figure.figsize'] = (4.5,2)
mpl.rcParams['figure.dpi'] = 100

existing_dir = 'en-figures'

In [6]:
def process_sheet(sheet_name):
    excel_raw_df = pd.read_excel('/Users/minghe/llm4faas/eva/_results/results_en_copy.xlsx', sheet_name=sheet_name)

    name_line = excel_raw_df.head(1).dropna(axis=1, how='any')
    pass_rate_df = excel_raw_df[excel_raw_df['Index'].str.contains('PASS RATE') == True].dropna(axis=1, how='any')
    compile_rate_df = excel_raw_df[excel_raw_df['Index'].str.contains('COMPILABLE RATE') == True].dropna(axis=1, how='any')

    raw_combined_df = pd.concat([name_line, pass_rate_df, compile_rate_df], ignore_index=True)

    # Get values
    models = raw_combined_df.iloc[0, 0:5].values
    pass_rate = raw_combined_df.iloc[1, 0:5].values * 100
    compilable_rate = raw_combined_df.iloc[2, 0:5].values * 100

    task_map = {
        'Remote Control': 'Remote Device Control',
        'Fixed Plans': 'Scheduled Plans',
        'Auto Adapt': 'Comfort Home',
        'Energy Saving': 'Energy Efficient Home',
    }

    model_map = {
        'gpt-4o': 'GPT-4o',
        'gpt-4o-mini': 'GPT-4o-mini',
        'gemini': 'Gemini',
        'llama3': 'Llama',
        'copilot': 'Copilot',
    }

    type_map = {
        'PASS RATE': 'Ready-to-Use Rate',
        'COMPILABLE RATE': 'Compile Rate',
    }


    mapped_models = [model_map.get(model, model) for model in models]
    task_name = task_map.get(sheet_name, sheet_name)


    # Add Task Name
    data = {
        'Model': list(mapped_models) * 2,
        'Rate': list(pass_rate) + list(compilable_rate),
        'Type': [type_map['PASS RATE']] * len(models) + [type_map['COMPILABLE RATE']] * len(mapped_models),
        'Task': task_name,
    }
    return pd.DataFrame(data)

# Sheet Names
sheet_names = ['Fixed Plans', 'Energy Saving', 'Remote Control', 'Auto Adapt']
all_dataframes = [process_sheet(sheet_name) for sheet_name in sheet_names]

# Data Frame containing all the data from the Excel
final_df_en = pd.concat(all_dataframes, ignore_index=True)
final_df_en['Language'] = 'English'

print(final_df_en)
final_df_en.to_csv('final_df_en.csv', index=False)

          Model        Rate               Type                   Task Language
0        GPT-4o   80.000000  Ready-to-Use Rate        Scheduled Plans  English
1   GPT-4o-mini   41.538462  Ready-to-Use Rate        Scheduled Plans  English
2        Gemini    9.230769  Ready-to-Use Rate        Scheduled Plans  English
3         Llama    0.000000  Ready-to-Use Rate        Scheduled Plans  English
4       Copilot   26.153846  Ready-to-Use Rate        Scheduled Plans  English
5        GPT-4o   86.153846       Compile Rate        Scheduled Plans  English
6   GPT-4o-mini   76.923077       Compile Rate        Scheduled Plans  English
7        Gemini   49.230769       Compile Rate        Scheduled Plans  English
8         Llama    0.000000       Compile Rate        Scheduled Plans  English
9       Copilot   43.076923       Compile Rate        Scheduled Plans  English
10       GPT-4o   38.095238  Ready-to-Use Rate  Energy Efficient Home  English
11  GPT-4o-mini   38.095238  Ready-to-Use Rate  Ener